In [ ]:
from IPython.utils import strdispatch
import os

In [ ]:
pip install mplcursors

In [ ]:
#%% Import necessary packages
from tkinter import YES
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from pyproj import CRS
import pickle
#os.environ["SHAPE_RESTORE.SHX"] = "YES"
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import mplcursors
import pandas as pd

import geopandas as gpd
import matplotlib.animation as animations
from numpy import reshape
from numpy.ma.core import std
from sklearn.preprocessing import MinMaxScaler
# Adjust plot font sizes
sns.set(font_scale=1.5)
# sns.set_style("white")
# os.chdir(os.path.join(et.io.HOME, "earth-analytics"))

In [ ]:
import pandas as pd
import os

ruta_base = '/content/drive/MyDrive/2.TPInteligenciaArtificial/Organizado/merged/'

# Ubicaciones
ubicaciones = ['Asuncion', 'Central', 'Ñeembucu', 'Concepcion']

def convertir_timestamp(df):
    métodos = [
        lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f'),
        lambda x: pd.to_datetime(x, format='ISO8601'),
        lambda x: pd.to_datetime(x, format='mixed'),
        lambda x: pd.to_datetime(x, format='mixed', dayfirst=True)
    ]

    for método in métodos:
        try:
            return método(df['timestamp'])
        except ValueError:
            continue

    raise ValueError("No se pudo convertir la columna timestamp con ningún método.")

def procesar_ubicacion(ubicacion):
    try:
        # Leer el archivo CSV unificado
        archivo = f'merged_{ubicacion}.csv'
        ruta_archivo = os.path.join(ruta_base, archivo)
        df = pd.read_csv(ruta_archivo)

        # Eliminar columnas latitude y longitude
        df = df.drop(columns=['latitude', 'longitude'])

        # Convertir la columna 'timestamp' a datetime
        df['timestamp'] = convertir_timestamp(df)

        # Manejar índices duplicados
        df = df.sort_values('timestamp').reset_index(drop=True)

        # Crear DataFrames para CG e IG
        cg_df = pd.DataFrame(index=df.index, columns=['positive', 'negative'])
        ig_df = pd.DataFrame(index=df.index, columns=['positive', 'negative'])

        # Llenar los DataFrames
        cg_mask = df['type'].isin([0, 40])
        ig_mask = df['type'] == 1

        cg_df.loc[cg_mask, 'positive'] = df.loc[cg_mask & (df['peakcurrent'] > 0), 'peakcurrent']
        cg_df.loc[cg_mask, 'negative'] = df.loc[cg_mask & (df['peakcurrent'] < 0), 'peakcurrent']

        ig_df.loc[ig_mask, 'positive'] = df.loc[ig_mask & (df['peakcurrent'] > 0), 'peakcurrent']
        ig_df.loc[ig_mask, 'negative'] = df.loc[ig_mask & (df['peakcurrent'] < 0), 'peakcurrent']

        # Rellenar NaN con 0
        cg_df = cg_df.fillna(0)
        ig_df = ig_df.fillna(0)

        # Añadir la columna timestamp a los DataFrames resultantes
        cg_df['timestamp'] = df['timestamp']
        ig_df['timestamp'] = df['timestamp']

        # Guardar los nuevos archivos
        cg_df.to_csv(os.path.join(ruta_base, f'CG_{ubicacion}_unificado.csv'), index=False)
        ig_df.to_csv(os.path.join(ruta_base, f'IG_{ubicacion}_unificado.csv'), index=False)

        print(f"Procesado {ubicacion}")
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo para {ubicacion}")
    except ValueError as e:
        print(f"Error al procesar {ubicacion}: {str(e)}")
        # Imprimir las primeras filas problemáticas
        print("Primeras filas problemáticas:")
        print(df['timestamp'].head())
    except Exception as e:
        print(f"Error inesperado al procesar {ubicacion}: {str(e)}")

# Procesar cada ubicación
for ubicacion in ubicaciones:
    procesar_ubicacion(ubicacion)

print("Procesamiento completado.")

Procesado Asuncion
Procesado Central
Procesado Ñeembucu
Procesado Concepcion
Procesamiento completado.


In [ ]:
df['timestamp'].head()

0    2016-01-01 00:33:35.956890106
1    2016-01-01 00:33:36.154273987
2    2016-01-01 00:49:14.815330029
3    2016-01-01 01:02:11.518081903
4    2016-01-01 01:14:21.134871006
Name: timestamp, dtype: object